1. 문서 내용 읽기 
2. 문서 분할(쪼개기)    
    -문서를 분할하지 않으면,
        -토큰 수 초과로 답변을 생성하지 못할 수 있음
        -문서 길이(input)가 길어서 답변 생성에 시간 소요됨 -> 답변 늦어짐 
3. 임베딩 - 벡터 데이터베이스에 저장
4. 질문이 있으면, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
## 패키지/라이브러리 설치 
# %pip install -qU docx2txt langchain_community langchain-text-splitters langchain-chroma

In [18]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import Docx2txtLoader
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter



# 1. 문서 내용 읽고 분할 =====================================================
loader = Docx2txtLoader('law_1.docx')

text_splitter = RecursiveCharacterTextSplitter( #분할된 문서 조각:청크
    chunk_size=1500,
    chunk_overlap=200,

)
document_list=loader.load_and_split(text_splitter=text_splitter) #실질적인 작업은 이 메써드 하나에서 실행 파일 읽고-> 분할-> text_splitter 기준으로 분할

## 2. 임베딩 -> 벡터 데이터베이스에 저장 =====================================================
## 2.1. 환경변수 읽어오기

load_dotenv()

# 2.2. 임베딩 모델 지정
embedding = OpenAIEmbeddings(model="text-embedding-3-large")


# 2.3. 백터 데이터베이스에 저장
## [방법 1] in memory -> RAM에 저장함.
database = Chroma.from_documents( #수치화된 분할된 문서를 chroma에 저장
    documents=document_list, #데이터:문서
    embedding=embedding,     #수치화

)
# 3. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색 =================================================================

# 3.1. 사용자 질문
query = '전세사기피해자 금융지원에 대해 설명해주세요.'

# 3.2. 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query) #앞쪽변수는 매개변수, 뒤쪽변수는 위에 사용자 질문을 저장한값 / 사용자의 정보중 유사한 정보검색을 하는것 -> retrieved docs에 list로 옴.

# 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 ===================================================
# 4.1. 프롬프트 작성
prompt = '''
[identity] 
- 당신은 전세사기피해 법률 전물가입니다.
- [context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs} 

Question: {query}
'''
## 4.2. 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(
    retrieved_docs=retrieved_docs,
    query=query
    ) 

## 4.3. LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm=ChatOpenAI(model='gpt-4o')

## 4.4. LLM 모델에 질문과 검색된 문서를 보냄
ai_message = llm.invoke(formatted_prompt)
ai_message


AIMessage(content='전세사기피해자에 대한 금융지원은 여러 가지 형태로 제공됩니다. 다음은 관련 법령에 따른 주요 지원 내용입니다.\n\n1. **융자 지원**: 전세사기 피해자와 긴박한 주거 안정이 필요한 자에게 국가는 필요한 자금을 융자할 수 있습니다. 이는 전세사기 피해자의 긴급한 주거 안정을 위해 국토교통부가 주관합니다.\n\n2. **우선변제 지원**: 「주택임대차보호법」 제8조에 따른 우선변제를 받지 못해 시급한 지원이 필요한 경우, 주택도시기금에서 필요 자금을 융자받을 수 있습니다.\n\n3. **신용정보 유예**: 전세사기 피해자의 보증금이 모두 변제되지 않은 경우, 금융사 등에서 해당 전세 관련 대출에 대한 채무 불이행 및 대위변제 등록을 유예하거나 삭제할 수 있습니다.\n\n4. **오피스텔 포함**: 주택저당채권대상에 주거목적으로 사용되는 오피스텔도 포함되며, 이것은 「한국주택금융공사법」 등 관련 법령에 의해 규정됩니다.\n\n이러한 금융지원 정책은 전세사기 피해자들이 최대한 빠르게 경제적 안정을 회복할 수 있도록 돕기 위한 것입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 299, 'prompt_tokens': 4167, 'total_tokens': 4466, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-BdtEz2Lidd